Let us go through the details related to upgrading Kafka Client on existing AWS EC2 Instance for Serverless Kafka setup using AWS Managed Service for Kafka (MSK). As of now we have deleted Serverless Kafka Cluster and created provisioned Kafka Cluster with Version 3.2.0. It is highly recommended to upgrade the client as well.

We need to ensure Java 11 is already setup, and then Kafka Client. As we have setup Java 11 already while configuring client to Serverless Kafka Cluster, we can proceed to setting up Kafka Client with 3.2.0 Version.

Once Kafka Client is setup we need to validate whether EC2 Instance can act as Kafka Client to the Provisioned Kafka Cluster.

In [1]:
!wget https://archive.apache.org/dist/kafka/3.2.0/kafka_2.12-3.2.0.tgz \
    -O /home/ec2-user/kafka_2.12-3.2.0.tgz

--2022-07-30 00:23:30--  https://archive.apache.org/dist/kafka/3.2.0/kafka_2.12-3.2.0.tgz
Resolving archive.apache.org (archive.apache.org)... 138.201.131.134, 2a01:4f8:172:2ec5::2
Connecting to archive.apache.org (archive.apache.org)|138.201.131.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104000785 (99M) [application/x-gzip]
Saving to: ‘/home/ec2-user/kafka_2.12-3.2.0.tgz’

100%[======================================>] 104,000,785 14.4MB/s   in 7.7s   

2022-07-30 00:23:38 (12.9 MB/s) - ‘/home/ec2-user/kafka_2.12-3.2.0.tgz’ saved [104000785/104000785]



In [3]:
!ls -ltr /home/ec2-user

total 101564
-rw-rw-r-- 1 ec2-user ec2-user 104000785 May 10 07:42 kafka_2.12-3.2.0.tgz
drwxrwxr-x 4 ec2-user ec2-user        66 Jul 29 01:07 mastering-msk
drwxrwxr-x 3 ec2-user ec2-user        35 Jul 29 06:30 gen-logs-python3


In [4]:
!sudo tar xzf ~/kafka_2.12-3.2.0.tgz -C /opt

In [5]:
!ls -ltr /opt

total 0
drwxr-xr-x 2 root     root       6 Aug 16  2018 rh
drwxr-xr-x 7 ec2-user ec2-user 105 Sep 14  2021 kafka_2.12-2.8.1
drwxr-xr-x 7 root     root     105 May  3 12:56 kafka_2.12-3.2.0
drwxr-xr-x 4 root     root      33 Jun 13 18:53 aws
lrwxrwxrwx 1 root     root      21 Jul 28 01:07 kafka -> /opt/kafka_2.12-2.8.1
drwxrwxr-x 5 ec2-user ec2-user 102 Jul 29 06:29 gen_logs


In [6]:
!sudo chown -R $USER:$USER /opt/kafka_2.12-3.2.0

In [11]:
!sudo unlink /opt/kafka

In [12]:
!sudo ln -s /opt/kafka_2.12-3.2.0 /opt/kafka

In [1]:
%env BS=b-2.retailcluster.871o9a.c22.kafka.us-east-1.amazonaws.com:9092,b-1.retailcluster.871o9a.c22.kafka.us-east-1.amazonaws.com:9092

env: BS=b-2.retailcluster.871o9a.c22.kafka.us-east-1.amazonaws.com:9092,b-1.retailcluster.871o9a.c22.kafka.us-east-1.amazonaws.com:9092


In [2]:
!/opt/kafka/bin/kafka-topics.sh \
    --bootstrap-server $BS \
    --list \
    --topic kafkademo

In [4]:
!/opt/kafka/bin/kafka-topics.sh \
    --bootstrap-server $BS \
    --create \
    --topic kafkademo \
    --partitions 2 \
    --replication-factor 2

# Default Replication Factor: 3
# Cluster Size: 2 Brokers
# Replication Factor cannot be greater than 2

Created topic kafkademo.


In [5]:
!/opt/kafka/bin/kafka-topics.sh \
    --bootstrap-server $BS \
    --list \
    --topic kafkademo

kafkademo


In [6]:
!/opt/kafka/bin/kafka-topics.sh \
    --bootstrap-server $BS \
    --describe \
    --topic kafkademo

Topic: kafkademo	TopicId: p3hAf13LS_qO8jOOXUMkLg	PartitionCount: 2	ReplicationFactor: 2	Configs: min.insync.replicas=2,message.format.version=3.0-IV1,unclean.leader.election.enable=true
	Topic: kafkademo	Partition: 0	Leader: 2	Replicas: 2,1	Isr: 2,1
	Topic: kafkademo	Partition: 1	Leader: 1	Replicas: 1,2	Isr: 1,2


In [8]:
!/opt/kafka/bin/kafka-topics.sh \
    --bootstrap-server $BS \
    --delete \
    --topic kafkademo